# MIMIC_Sepsis

## 1. Preparation

To run this document the following requirements must be satisfied:

- Implement the database mimic in **PostgreSQL** and start it. The instruction can be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres). (The name of this environment should be **mimiciv**)
- generate useful abstractions of raw MIMIC-IV data. The instruction be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/concepts_postgres) 



To install all the libraries, run:
```
pip install psycopg2 csv pandas
```


After all the preparation is done, run the following cell to set the connection to the database.
 

In [1]:
import psycopg2
from psycopg2 import sql
import csv
import pandas as pd
import os
import shutil
import csv

# implement the username, password and database name
conn = psycopg2.connect(host='', user='', password='', database='mimiciv')

## 2. Extract selected data from the original database 

According to the paper we need to extract the **state space** and **action space** respectively from the mimiciv database. The table **mimic4 itemid.csv** lists all the items required.

* Here the *IV fluid bolus* of the action space is removed as no data could be found.

#### **Uncomment the following line if you first time run the code**

In [11]:
# uncomment the following line if you first time run the code

# Read the SQL file

try:
    with open('sql_file/selection_patients_cohort.sql', 'r') as file0:
        sql_script_selection_patients_cohort = file0.read()

    with open('sql_file/action_from_inputevents.sql', 'r') as file1:
        sql_script_action = file1.read()
        
    with open('sql_file/chartevents_dataneeded.sql', 'r') as file2:
        sql_script_state = file2.read()

    # Execute the SQL script
    cursor = conn.cursor()
    
    # cursor.execute(sql.SQL(sql_script_selection_patients_cohort))
    cursor.execute(sql.SQL(sql_script_action))
    # cursor.execute(sql.SQL(sql_script_state))

    conn.commit()
    cursor.close()
    
    
except (Exception, psycopg2.DatabaseError) as error:
    print("Error executing SQL statement:", error)

In [12]:
with conn.cursor() as cursor:
    command = "SELECT distinct stay_id FROM mimiciv_derived.sepsis_patients_cohort ;"
    cursor.execute(command)   
    result = cursor.fetchall()
    stay_ids= [row[0] for row in result]
    num_stay_ids = len(stay_ids)
    print('Number of stay_ids: ' + str(num_stay_ids))
    cursor.close()

Number of stay_ids: 6669


## 3. Data transfor
Here we first deal with the action data. 
TODO: here the value per minute is computed, do we need this? (paper4.1,P5)

In [33]:
# generate the dictionary of action
with open('csv/itemid_label_action.csv', newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)
    # Skip the header row
    next(reader)
    # Initialize an empty dictionary and list
    action_label = {}
    a_itemid_list = []
    # Iterate over the rows in the CSV file
    for row in reader:
        # Add the key-value pair to the dictionary
        action_label[row[0]] = row[1]
        # Add the itemid to the list
        a_itemid_list.append(row[0])

if os.path.exists('./output/action/IV_fluid_bolus'):shutil.rmtree('./output/action/IV_fluid_bolus')
if os.path.exists('./output/action/vasopressors'):shutil.rmtree('./output/action/vasopressors')
os.makedirs('./output/action/IV_fluid_bolus')
os.makedirs('./output/action/vasopressors')

with conn.cursor() as cursor:

    for itemid in a_itemid_list:
        # QUESTION: why do we need to order by starttime?
        command = "select stay_id, starttime, endtime, amount from mimiciv_derived.sepsis_action where itemid={} order by starttime;".format(itemid)
        cursor.execute(command)

        result = cursor.fetchall()
        df = pd.DataFrame(result)
        df.columns = ['stay_id', 'starttime', 'endtime', 'amount']
        
        df['duration'] = df['endtime'] - df['starttime']
        df['duration'] = df['duration'].dt.total_seconds()  # Convert duration to seconds
        df['duration'] = df['duration'] / 60
        df['value_per_minute'] = df['amount'] / df['duration']
        
        if "Dextrose_5%" in action_label[str(itemid)] or "NaCl_0_9%" in action_label[str(itemid)]:
            df.to_csv('./output/action/IV_fluid_bolus/{}.csv'.format(action_label[str(itemid)]), index=0)
            print("output action (IV_fluid_bolus):\t"+action_label[str(itemid)]+".csv")
        else:
            df.to_csv('./output/action/vasopressors/{}.csv'.format(action_label[str(itemid)]), index=0)
            print("output action (vasopressors):\t"+action_label[str(itemid)]+".csv")
    cursor.close()
    

output action (IV_fluid_bolus):	NaCl_0_9%.csv
output action (IV_fluid_bolus):	Dextrose_5%.csv
output action (vasopressors):	Norepinephrine.csv
output action (vasopressors):	Vasopressin.csv
output action (vasopressors):	Dobutamine.csv
output action (vasopressors):	Milrinone.csv
output action (vasopressors):	Phenylephrine.csv
output action (vasopressors):	Dopamine.csv
output action (vasopressors):	Epinephrine.csv


### get IV fluid bolus statistical data

In [38]:
# 文件夹路径
folder_path = 'output/action/IV_fluid_bolus'
print('项目\tIV fluid bolus\tCount')

# 获取文件夹中的所有CSV文件路径
file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

index = 0
# 遍历每个CSV文件，获取行数
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    with open(file_path, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        rows = sum(1 for row in csv_reader)
        index += 1
    print(f"{index}\t{file_path[29:-4]}\t{rows}")


项目	IV fluid bolus	Count
1	Dextrose_5%	144445
2	NaCl_0_9%	183559


### get Vasopressors statistical data

In [11]:
# 文件夹路径
folder_path = 'output/action/vasopressors'
print('项目\tVasopressors\tCount\t占比\t前n项占比')

# 获取文件夹中的所有CSV文件路径
file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# 存储每个CSV文件的行数
rows_dict = {}

# 计算行数之和
total_rows = 0

# 遍历每个CSV文件，获取行数
for file_path in file_paths:
    with open(file_path, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        rows = sum(1 for row in csv_reader)
        rows_dict[file_path] = rows
        total_rows += rows

# 按行数大小对字典进行排序
sorted_rows = sorted(rows_dict.items(), key=lambda x: x[1], reverse=True)

total_rows_first_n = 0
index = 0
# 打印排序结果
for file_path, rows in sorted_rows:
    index += 1
    total_rows_first_n += rows
    print(f"{index}\t{file_path[27:-4]}\t{rows}\t{round(rows/total_rows*100,2)}%\t{round(total_rows_first_n/total_rows*100,2)}%")
print(f"Total\t\t\t{total_rows}\t100%\t100%")

项目	血管加压药	Count	占比	前n项占比
1	Norepinephrine	60081	66.46%	66.46%
2	Phenylephrine	19667	21.76%	88.22%
3	Vasopressin	3763	4.16%	92.38%
4	Epinephrine	2829	3.13%	95.51%
5	Dopamine	2448	2.71%	98.22%
6	Dobutamine	1132	1.25%	99.47%
7	Milrinone	481	0.53%	100.0%
Total			90401	100%	100%


前5项血管加压药占比:98.22%，可见前5项血管加压药占比较大，因此我们只需要考虑前5项血管加压药即可。忽略Dobutamine和Milrinone。

后两项血管加压药也被此文献忽视。'Vasopressor dose equivalence: A scoping review and suggested formula by Goradia et al. 2020'

前5项血管加压药的等效计量值可以直接从mimiciv_derived.norepinephrine_equivalent_dose中获取:

In [28]:
# 从mimiciv_derived.norepinephrine_equivalent_dose中获取前5项血管加压药的等效计量值 norepinephrine_equivalent_dose_rate

with conn.cursor() as cursor:

    command = "select stay_id, starttime, endtime, norepinephrine_equivalent_dose from mimiciv_derived.norepinephrine_equivalent_dose where stay_id in (select stay_id from mimiciv_derived.sepsis_patients_cohort);" # get norepinephrine_equivalent_dose_rate
    cursor.execute(command)

    result = cursor.fetchall()
    df = pd.DataFrame(result)
    df.columns = ['stay_id', 'starttime', 'endtime', 'norepinephrine_equivalent_dose_rate'] # norepinephrine_equivalent_dose in mcg/kg/min
    
    df['duration'] = df['endtime'] - df['starttime']
    df['duration'] = df['duration'].dt.total_seconds()  # Convert duration to seconds
    df['duration'] = df['duration'] / 60
    df['norepinephrine_equivalent_dose_rate'] = df['norepinephrine_equivalent_dose_rate'].astype(float)
    
    df.to_csv('./output/action/vasopressors_norepinephrine_equivalent_dose.csv', index=0)
    print("output action (vasopressors_norepinephrine_equivalent_dose.csv)")

    cursor.close()

# 文件夹路径
folder_path = 'output/action'
print('\n项目\tVasopressors_norepinephrine_equivalent_dose\tCount')

# 获取文件夹中的所有CSV文件路径
file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

index = 0
# 遍历每个CSV文件，获取行数
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    with open(file_path, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        rows = sum(1 for row in csv_reader)
        index += 1
    print(f"{index}\t{file_path[14:-4]}\t{rows}")

output action (vasopressors_norepinephrine_equivalent_dose.csv)

项目	Vasopressors_norepinephrine_equivalent_dose	Count
1	vasopressors_norepinephrine_equivalent_dose	91823


Then, we treat the **state space** with similar method.

In [5]:
threshold = 1000
itemid_list_state=[]
# generate the dictionary of itemid-abbr
with open('csv/itemid_label_state.csv', newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)
    # Skip the header row
    next(reader)
    # Initialize an empty dictionary and list
    label_state = {}
    itemid_list = []
    # Iterate over the rows in the CSV file
    for row in reader:
        # Add the key-value pair to the dictionary
        label_state[row[0]] = row[1]
        # Add the itemid to the list
        itemid_list.append(row[0])

# Execute the SQL command
with conn.cursor() as cursor:
    
    for itemid in itemid_list:
        
        command_count = "select count(distinct(stay_id)) from mimiciv_derived.sepsis_state where itemid={};".format(itemid)
        cursor.execute(command_count)
        num = cursor.fetchone()[0]
        
        command = "select stay_id, charttime, valuenum from mimiciv_derived.sepsis_state where itemid={} order by charttime;".format(itemid)
        cursor.execute(command)   
        result = cursor.fetchall()
        df=pd.DataFrame(result)
        try:
            df.columns = ['stay_id', 'charttime', 'valuenum']
            df.astype({'stay_id': int, 'charttime': 'datetime64[ns]', 'valuenum': float})
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error executing SQL statement:", error) 
        os.makedirs('./output/state', exist_ok=True)
        
        if (num<num_stay_ids/threshold):
            print("drop:{0:40}".format(label_state[str(itemid)]+".csv")+"\tnumber of stay_id:"+str(num))
            
        else:
            df.to_csv('./output/state/{}.csv'.format(label_state[str(itemid)]),index=0)
            itemid_list_state.append(itemid)
            print("output:{0:40}".format(label_state[str(itemid)]+".csv")+"\tnumber of stay_id:"+str(num))
        
        
        
    cursor.close()
print(itemid_list_state)

output:Heartrate.csv                           	number of stay_id:6669
output:ABPs.csv                                	number of stay_id:2129
output:NBPs.csv                                	number of stay_id:6632
output:ABPd.csv                                	number of stay_id:2131
output:NBPd.csv                                	number of stay_id:6632
output:ABPm.csv                                	number of stay_id:2168
output:NBPm.csv                                	number of stay_id:6632
output:RespiratoryRate.csv                     	number of stay_id:6669
output:TemperatureF.csv                        	number of stay_id:6576
output:TemperatureC.csv                        	number of stay_id:832
output:PH_A.csv                                	number of stay_id:3663
output:PH_V.csv                                	number of stay_id:3184
output:ABE.csv                                 	number of stay_id:3645
output:Hematocrit_serum.csv                    	number of stay_id:6596
output:

## 4.hourly sample

In [8]:
def hourly_resample(selected_id):
    # Set the folder path where the CSV files are stored
    folder_path = './output/state/'
    columns=['chartdatetime']

    # define a new dataframe
    df_output = pd.DataFrame(columns=columns)

    # selected_id = 32768012
    #selected_id = 32950566

    #FIXME: flag only for test
    i=0

    # Loop through the file paths and read each file into a DataFrame
    for itemid in itemid_list_state:
        
        feature=label_state[str(itemid)]
        path = folder_path + feature+'.csv'
        
        # Load the CSV file into a pandas DataFrame
        dtypes={'stay_id':str,'chartdatetime':str,feature:str}
        df = pd.read_csv(path,names=['stay_id', 'chartdatetime', feature ],dtype=dtypes)
        df['stay_id'] = pd.to_numeric(df['stay_id'], errors='coerce')
        
        # Filter the DataFrame to only the rows from the selected stay_id
        df_filtered = df[df['stay_id'] == selected_id]
        
        # Convert the 'datetime' column to a datetime object
        df_filtered.loc[:,'chartdatetime'] = pd.to_datetime(df_filtered['chartdatetime'].copy())
        #df_filtered['chartdatetime'] = df_filtered['chartdatetime'].apply(pd.to_datetime)

        # Set the 'datetime' column as the DataFrame's index
        df_filtered.set_index('chartdatetime', inplace=True)
        
        # # Resample the DataFrame hourly and forward fill missing values
        df_hourly= df_filtered.resample('H').ffill()
        df_hourly=df_hourly.drop(['stay_id'],axis=1)
        
        df_output['chartdatetime'] = pd.to_datetime(df_output['chartdatetime'])
        #df_output = pd.concat([df_output, df_hourly], join="outer",sort=False)
        df_output=pd.merge(df_output,df_hourly,how='outer',on='chartdatetime')

        
        i+=1
        if(i==50): break

    os.makedirs('./output/hourly_sample/', exist_ok=True)
    df_output.reset_index().to_csv(f'./output/hourly_sample/stay_id{selected_id}.csv',index=0)

    # print(i)

    # # Reset the index and save the resampled DataFrame to a new CSV file
    # df_hourly.reset_index().to_csv('./output/your_resampled_file.csv', index=False, header=None)  

### Loop hourly_resample through stay_ids

In [ ]:
for selected_id in stay_ids:
    hourly_resample(selected_id)